# Izračun točk

In [ ]:
import pandas as pd
import re

In [ ]:
benchmarks = pd.read_csv("benchmarks.csv", names=["name", "sudoku", "result", "time"])

In [ ]:
def kind(sudoku):
    match = re.search("(?P<kind>\w+)-\d+\.sdk", sudoku)
    if match:
        return match.group("kind")


benchmarks["kind"] = benchmarks.sudoku.apply(kind)

In [ ]:
time_range = (
    benchmarks[benchmarks.result == "CORRECT"]
    .groupby("sudoku")
    .aggregate({"time": [("fastest", "min"), ("slowest", "max")]})
)
time_range.columns = time_range.columns.get_level_values(1)
benchmarks = pd.merge(benchmarks, time_range, on="sudoku", how="left")

In [ ]:
incorrect = benchmarks.result != "CORRECT"
benchmarks.loc[incorrect, "time"] = benchmarks[incorrect].slowest * 2

benchmarks["relative_time"] = benchmarks.time / benchmarks.fastest

In [ ]:
benchmarks

## Točke za pravilnost

- 5 točk - 100% pravilno rešenih običajnih sudokujev
- 4 točke - ≥50% pravilno rešenih običajnih sudokujev
- 3 točke - >0% pravilno rešenih običajnih sudokujev
- 0-2 točki - odvisno od pravilnosti pomožnih funkcij - se določi ročno

In [ ]:
number_of_correct = (
    benchmarks[(benchmarks.kind == "obicajni") & (benchmarks.result == "CORRECT")]
    .groupby("name")
    .size()
)

In [ ]:
number_of_correct.sort_values(ascending=False).plot.bar()

In [ ]:
total = len(set(benchmarks[benchmarks.kind == "obicajni"].sudoku))


def tocke_za_pravilnost(correct):
    if correct == total:
        return 5
    elif correct / total >= 0.5:
        return 4
    else:
        # če se vrednost pojavi, je moral biti vsaj en pravilen
        return 3

In [ ]:
number_of_correct.apply(tocke_za_pravilnost)

## Točke za hitrost

In [ ]:
speed_benchmarks = benchmarks[
    (benchmarks.kind == "obicajni") & ~pd.isna(benchmarks.time)
]

In [ ]:
speed_benchmarks.boxplot("relative_time", by="name", vert=False)

In [ ]:
speed = speed_benchmarks.groupby("name").mean().relative_time
speed.sort_values().plot.bar(log=True)
speed

## Točke za razširitve



In [ ]:
extension_benchmarks = benchmarks[
    (benchmarks.result != "ERROR") & (benchmarks.kind != "obicajni")
]

In [ ]:
extension_benchmarks.groupby(["name", "kind", "result"]).size().unstack()

In [ ]:
extension_benchmarks.groupby(["name", "kind"]).mean().relative_time